In [1]:
#!python --version
#nötig ist version 3.8.17

## Bibliotheken, die ich benutze

In [2]:
import cv2 as cv          #zugriff auf kamera
import mediapipe as mp    #model erstellen
import numpy as np        #mit arrays arbeiten
import os                 #mit dem Dateisystem interagieren

ModuleNotFoundError: No module named 'cv2'

## Verzeichnis für datensatz/"videos" anlegen

In [46]:
#https://www.geeksforgeeks.org/create-a-directory-in-python/

#ornder name für meine datensätze (daten werden als ".npy" dateiformat gespeichert (als arrays))
DATA_PATH = os.path.join("datensatz1")

#gebärden die wir aufnehmen
gesten = np.array(["deutschland, frieden, hallo, liebe, praesentation, wettbewerb"])

#anzahl der "videos" pro gebärde
num_videos = 40

#länge der video (in frame)
vid_length = 20

In [47]:
#ordner erstellen
for geste in gesten:
    for vid in range(num_videos):
        os.makedirs(os.path.join(DATA_PATH, geste, str(vid)))

## auf unsere Webcam zugreifen + Modell/K-Punkte und Videos erstellen

In [5]:
#ich verwende Mediapipe, aber gute Alternativen sind OpenPose und AlphaPose

#hilfreiche links:
#https://developers.google.com/mediapipe/solutions/vision/hand_landmarker/python
#https://developers.google.com/mediapipe/solutions/vision/gesture_recognizer/python
#https://developers.google.com/mediapipe/solutions/vision/pose_landmarker/python#image
#https://developers.google.com/mediapipe/solutions/vision/holistic_landmarker
#https://www.youtube.com/watch?v=01sAkU_NvOY&ab_channel=freeCodeCamp.org
#https://www.youtube.com/watch?v=doDUihpj6ro&list=PLyx6mI41rWldgZWHzHBGBQfP5MxFQNcFf&index=5&ab_channel=NicholasRenotte

In [6]:
#modelle erstellen

mp_holistic = mp.solutions.holistic
#holistic kombiert hände, körper und gesicht erkennung
#https://developers.google.com/mediapipe/solutions/vision/holistic_landmarker
#https://github.com/google/mediapipe/blob/master/docs/solutions/holistic.md

mp_drawing = mp.solutions.drawing_utils

In [7]:
#farben umwandlung + model hinzufügen
def conversion(frame, model):
    frame = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
    #hier umwandeln wir frame in rgb weil mediapipe braucht rgb format
    
    frame.flags.writeable = False
    results = model.process(frame)
    #hier hinzufügen wir mediapipe model/"eine Vorhersage treffen"
    
    frame.flags.writeable = True
    frame = cv.cvtColor(frame, cv.COLOR_RGB2BGR)
    #hier zurückkehren wir in BGR damit cv2 es zeigen kann
    
    return frame, results

In [8]:
#orientierungspunkte zeigen lassen an cv
def show_kpoints(main_frame, results):
    #wir werden gesichtspunkte nicht gebrauchen bei gebärdensprache
    #mp_drawing.draw_landmarks(main_frame, results.face_landmarks, mp_holistic.FACE_CONNECTIONS)
    
    #                         image       orientierungspunkte     verbindungskarte
    mp_drawing.draw_landmarks(main_frame, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
    mp_drawing.draw_landmarks(main_frame, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(main_frame, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

In [9]:
#k-points extrahieren

#wir verwenden die "try, except"-Methode, falls nicht alle Schlüsselpunkte in der Kamera angezeigt werden, so dass es nicht zu einem Fehler kommt

def extract_keypoints(results):
    try:
        pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten()
    except:
        pose = np.zeros(33*4)

    try:
        linke_hand = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten()
    except:
        linke_hand = np.zeros(21*3)

    try:
        rechte_hand = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten()
    except:
        rechte_hand = np.zeros(21*3)
    
    return np.concatenate([pose, linke_hand, rechte_hand])

In [48]:
#https://www.youtube.com/watch?v=oXlwWbU8l2o&ab_channel=freeCodeCamp.org

capture = cv.VideoCapture(0)
# 1 = mein computer webcam
# 0 = mein externe webcam/handy

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    for geste in gesten:
    #schleife durch geste
        for vid in range(num_videos):
        #schleife durch videos
            for frame_num in range(vid_length):
            #schleife durch video frames

                isTrue, frame = capture.read()

                flipped_frame = cv.flip(frame, 1)   #bild spiegeln

                main_frame, results = conversion(flipped_frame, holistic)
                #hier hinzufügen wir unser model

                #orientierungspunkte in kamera zeigen lassen
                show_kpoints(main_frame, results)
                                 
                #frames sammeln 
                if frame_num == 0: 
                                            #text;                text position
                    cv.putText(main_frame, 'START DER SAMMLUNG', (110,240),
                               cv.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 255), 3, cv.LINE_AA)
                                #schrift; schriftgröße; schriftfarbe(BGR); linienstärke; linetype(schönere aussehen)
                                #https://docs.opencv.org/4.x/dc/da5/tutorial_py_drawing_functions.html
                        
                    cv.putText(main_frame, 'Sammeln von Frames fuer "{}"; Video Nr.{}'.format(geste, vid), (10,22), 
                               cv.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2, cv.LINE_AA)
                    
                    cv.imshow("Kamera - Gestus", main_frame)  #Anzeigen, Wiedergeben des Bildes
                    cv.waitKey(2000)
                        
                else: 
                    cv.putText(main_frame, 'Sammeln von Frames fuer "{}"; Video Nr.{}'.format(geste, vid), (10,22), 
                                 cv.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2, cv.LINE_AA)
                    
                    cv.imshow("Kamera - Gestus", main_frame)  #Anzeigen, Wiedergeben des Bildes

                
                #k-point in Ordner extrahieren
                
                #k-points speichern in variable
                kpoints = extract_keypoints(results)
                
                #weg/path beschreiben für k-points
                npy_path = os.path.join(DATA_PATH, geste, str(vid), str(frame_num))
                
                #k-points in folder speichern
                np.save(npy_path, kpoints)
                

                if cv.waitKey(20) & 0xFF==ord("s"):
                    break
                    #mit "s"(stop), stoppt videoaufnahme    

    capture.release()
    cv.destroyAllWindows()

In [17]:
#cv fenster ausschalten (im absturz fall)

capture.release()
cv.destroyAllWindows()

# Quellen und Hilfreiche links

https://www.youtube.com/watch?v=doDUihpj6ro&list=PLyx6mI41rWldgZWHzHBGBQfP5MxFQNcFf&index=5&ab_channel=NicholasRenotte
https://www.youtube.com/watch?v=01sAkU_NvOY&ab_channel=freeCodeCamp.org
https://www.youtube.com/watch?v=oXlwWbU8l2o&ab_channel=freeCodeCamp.org
https://www.geeksforgeeks.org/create-a-directory-in-python/
https://developers.google.com/mediapipe/solutions/vision/hand_landmarker/python
https://developers.google.com/mediapipe/solutions/vision/gesture_recognizer/python
https://developers.google.com/mediapipe/solutions/vision/pose_landmarker/python#image
https://developers.google.com/mediapipe/solutions/vision/holistic_landmarker
https://developers.google.com/mediapipe/solutions/vision/holistic_landmarker
https://github.com/google/mediapipe/blob/master/docs/solutions/holistic.md
https://docs.opencv.org/4.x/dc/da5/tutorial_py_drawing_functions.html